In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix
from concurrent.futures import ThreadPoolExecutor, as_completed
import umap
import random
import multiprocessing
random.seed(123)
from joblib import Parallel, delayed
import warnings
import os
from tqdm import tqdm
import math
from PIL import Image, ImageDraw, ImageFont
import gc
import re
import multiprocessing


# by cell types

In [5]:
files = os.listdir("h5ad_harmony/")
QC_table=pd.read_csv('QC-SF4 - HARMONY-V1.csv')
QC_table.head()

,AIFI_L3,leiden,Notes
0,ASDC,12,B doublets
1,ASDC,6,T doublets
2,Activated memory B cell,11,Monocytes doublets
3,Activated memory B cell,1,Transitional B cells
4,Activated memory B cell,7,Monocytes doublets


In [ ]:
%%time

def process_file(file, QC_table):
    print("Reading: "+file)
    adata_clustered = sc.read_h5ad('h5ad_harmony/' + file)
    adata_clustered=adata_clustered.raw.to_adata() 
    adata_clustered.raw=adata_clustered
    filtered_leiden = QC_table[QC_table['AIFI_L3'] == adata_clustered.obs['celltypist_l3'][0]]['leiden'].tolist()
    adata_clustered = adata_clustered[~adata_clustered.obs['leiden'].isin([str(i) for i in filtered_leiden])]
    adata_clustered.write_h5ad('h5_cleaned_by_celltype/' + adata_clustered.obs['celltypist_l3'][0] + '.h5ad')

with multiprocessing.Pool(processes=30) as pool:
    results = pool.starmap(process_file, [(f, QC_table) for f in files])
